# Projeto de Férias - Aprendizado de Máquina
-----------------------

# Predição de emissão de gases em um dataset sobre turbinas 

**Aluno:** Enzo J. Xavier - RM 24035

**Orientador:** Dr. Daniel Roberto Cassar

## Introdução:

Relembrar: Explicar notebook anterior, referenciar link 

Explicar: Falar objetivo do notebook atual, passo a passo

Modelar: Abordar os modelos usados, teoria e fórmula

## Importações e definições:

Importando as principais bibliotecas usadas neste notebok. A documentação de cada biblioteca se encontra no final do arquivo, na sessão "Referências"

In [9]:
import os
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Torch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import lightning as L
import darts


In [8]:
class DataModule(L.LightningDataModule):
    def __init__(self):
        super().__init__()

    def setup(self, stage, atributos, target):
        """Aqui devemos alterar o estado da classe para adicionar as informações 
        referentes aos conjuntos de treino, teste e validação. O argumento `stage` 
        deve existir e ele indica em qual estágio o processo de treino está 
        (pode ser `fit` para treinamento/validação e `test` para teste).

        É nesta etapa onde aplicamos transformações aos dados caso necessário."""

        # Diretório
        atual = os.getcwd()
        caminho = os.path.dirname(atual)
        os.chdir(f'{caminho}\\Datasets')
        
        # Carregar datasets
        df_2011 = pd.read_csv('gt_2011.csv')
        df_2012 = pd.read_csv('gt_2012.csv')
        df_2013 = pd.read_csv('gt_2013.csv')
        df_2014 = pd.read_csv('gt_2014.csv')
        df_2015 = pd.read_csv('gt_2015.csv')

        df_treino = pd.concat([df_2011,df_2012], ignore_index= True)
        df_val = df_2013
        df_teste = pd.concat([df_2014,df_2015], ignore_index= True)
        df_tot = pd.concat([df_2011,df_2012,df_2013,df_2014,df_2015], ignore_index = True)

        # Coluna de data
        df_2011['Data'] = df_2011.index
        df_2012['Data'] = df_2012.index
        df_2013['Data'] = df_2013.index
        df_2014['Data'] = df_2014.index
        df_2015['Data'] = df_2015.index

        df_treino['Data'] = df_treino.index
        df_teste['Data'] = df_teste.index
        df_tot['Data'] = df_tot.index

        X_treino = df_treino.reindex(atributos, axis=1).values
        y_treino = df_treino.reindex(target, axis=1).values

        self.x_scaler = StandardScaler()
        self.x_scaler.fit(X_treino)

        self.y_scaler = StandardScaler()
        self.y_scaler.fit(y_treino)

        if stage == "fit":
            X_val = df_val.reindex(atributos, axis=1).values
            y_val = df_val.reindex(target, axis=1).values

            X_treino = self.x_scaler.transform(X_treino)
            y_treino = self.y_scaler.transform(y_treino)

            X_val = self.x_scaler.transform(X_val)
            y_val = self.y_scaler.transform(y_val)

            self.X_treino = torch.tensor(X_treino, dtype=torch.float32)
            self.y_treino = torch.tensor(y_treino, dtype=torch.float32)

            self.X_val = torch.tensor(X_val, dtype=torch.float32)
            self.y_val = torch.tensor(y_val, dtype=torch.float32)

        if stage == "test":
            X_teste = df_teste.reindex(atributos, axis=1).values
            y_teste = df_teste.reindex(target, axis=1).values

            X_teste = self.x_scaler.transform(X_teste)
            y_teste = self.y_scaler.transform(y_teste)

            self.X_teste = torch.tensor(X_teste, dtype=torch.float32)
            self.y_teste = torch.tensor(y_teste, dtype=torch.float32)

    def train_dataloader(self):
        return DataLoader(TensorDataset(self.X_treino, self.y_treino))

    def val_dataloader(self):
        return DataLoader(TensorDataset(self.X_val, self.y_val))

    def test_dataloader(self):
        return DataLoader(TensorDataset(self.X_teste, self.y_teste))

### Darts:

### Pytorch

In [ ]:
class RegressorRNN(nn.Module):
    def __init__(self, num_atributos, num_neurons_por_camada, num_camadas):
        super().__init__()

        self.rnn = nn.RNN(
            num_atributos,
            num_neurons_por_camada,
            num_camadas,
            batch_first=True,
        )
        self.cam_saida = nn.Linear(num_neurons_por_camada, 1)

    def forward(self, x):
        _, oculta = self.rnn(x)
        out = self.cam_saida(oculta[-1])
        out = out.view(-1)
        return out

In [10]:

# Lightning Module for LSTM Model
class TimeSeriesModel(L.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim=2):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        return self.fc(hidden[-1])

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs.unsqueeze(-1))
        loss = torch.nn.functional.mse_loss(outputs, targets)
        self.log('train_loss', loss)
        return loss

In [ ]:
# Create DataLoader
    dataset = StockDataset(seq_length=10)
    train_loader = DataLoader(dataset, batch_size=12, shuffle=True)

    # Initialize model
    model = TimeSeriesModel()

    # Train model
    trainer = L.Trainer(max_epochs=200)
    trainer.fit(model, train_loader)

In [ ]:
treinador = L.Trainer(max_epochs=NUM_EPOCAS)

num_dados_de_entrada = 3
num_dados_de_saida = 1
camadas_ocultas = [3, 2, 2]
funcao_ativacao = nn.Sigmoid()

modelo_mlp = MLP2(
    num_dados_de_entrada, camadas_ocultas, funcao_ativacao, num_dados_de_saida
)

dm = DataModule()

treinador.fit(modelo_mlp, dm)

### Conclusão

### Referências

[1]

[2]

[3]

[4]

[] Biblioteca do normalizador padrão - scikit learn: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

[] David Menotti, aula sobre MLP: https://www.inf.ufpr.br/menotti/ci171-182/slides/ci171-classMLP.pdf